In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

pd.options.mode.chained_assignment = None

In [2]:
# Import data
df = pd.read_csv("ripa_stops_datasd.csv", low_memory=False)

In [3]:
df.head()

,stop_id,ori,agency,exp_years,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,assignment,...,beat_name,pid,isstudent,perceived_limited_english,perceived_age,perceived_gender,gender_nonconforming,gend,gend_nc,perceived_lgbt
0,2443,CA0371100,SD,10,2018-07-01,00:01:37,30,0,1,"Patrol, traffic enforcement, field operations",...,Pacific Beach 122,1,0,0,25,Male,0,1,NaN,No
1,2444,CA0371100,SD,18,2018-07-01,00:03:34,10,0,1,"Patrol, traffic enforcement, field operations",...,Mission Beach 121,1,0,0,25,Male,0,1,NaN,No
2,2447,CA0371100,SD,1,2018-07-01,00:05:43,15,1,10,Other,...,El Cerrito 822,1,0,0,30,Male,0,1,NaN,No
3,2447,CA0371100,SD,1,2018-07-01,00:05:43,15,1,10,Other,...,El Cerrito 822,2,0,0,30,Female,0,2,NaN,No
4,2448,CA0371100,SD,3,2018-07-01,00:19:06,5,0,1,"Patrol, traffic enforcement, field operations",...,Ocean Beach 614,1,0,0,23,Male,0,1,NaN,No


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464044 entries, 0 to 464043
Data columns (total 29 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   stop_id                    464044 non-null  int64  
 1   ori                        464044 non-null  object 
 2   agency                     464044 non-null  object 
 3   exp_years                  464044 non-null  int64  
 4   date_stop                  464044 non-null  object 
 5   time_stop                  464044 non-null  object 
 6   stopduration               464044 non-null  int64  
 7   stop_in_response_to_cfs    464044 non-null  int64  
 8   officer_assignment_key     464044 non-null  int64  
 9   assignment                 464044 non-null  object 
 10  intersection               43743 non-null   object 
 11  address_block              417070 non-null  float64
 12  land_mark                  56 non-null      object 
 13  address_street             44

In [5]:
###### Test ######
df[['intersection','address_block','land_mark','address_street']][50:60]

,intersection,address_block,land_mark,address_street
50,NaN,2700.0,NaN,Beyer
51,NaN,900.0,NaN,5th ave
52,NaN,800.0,NaN,05th
53,NaN,700.0,NaN,05TH AVENUE
54,NaN,500.0,NaN,e st
55,NaN,900.0,NaN,garnet ave
56,NaN,1700.0,NaN,SAN DIEGO
57,NaN,5500.0,NaN,Miramar Rd
58,NaN,4200.0,NaN,CLAIREMONT MESA BLVD
59,NaN,4100.0,NaN,ocean front walk


## Getting columns we're interested in

In [6]:
# Get reduced dataframe with relevant info
rdf= df[['stop_id',
         'beat',
         'beat_name',
         'isschool',
         'date_stop', 
         'time_stop', 
         'stopduration',
         'stop_in_response_to_cfs', # cfs == call for service
         'officer_assignment_key',
         'assignment',
         'exp_years', 
         'pid',
         'isstudent',
         'perceived_limited_english',
         'perceived_age',
         'perceived_gender','perceived_lgbt']]

In [7]:
rdf.head()

,stop_id,beat,beat_name,isschool,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,assignment,exp_years,pid,isstudent,perceived_limited_english,perceived_age,perceived_gender,perceived_lgbt
0,2443,122,Pacific Beach 122,0,2018-07-01,00:01:37,30,0,1,"Patrol, traffic enforcement, field operations",10,1,0,0,25,Male,No
1,2444,121,Mission Beach 121,0,2018-07-01,00:03:34,10,0,1,"Patrol, traffic enforcement, field operations",18,1,0,0,25,Male,No
2,2447,822,El Cerrito 822,0,2018-07-01,00:05:43,15,1,10,Other,1,1,0,0,30,Male,No
3,2447,822,El Cerrito 822,0,2018-07-01,00:05:43,15,1,10,Other,1,2,0,0,30,Female,No
4,2448,614,Ocean Beach 614,0,2018-07-01,00:19:06,5,0,1,"Patrol, traffic enforcement, field operations",3,1,0,0,23,Male,No


In [8]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464044 entries, 0 to 464043
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   stop_id                    464044 non-null  int64 
 1   beat                       464044 non-null  int64 
 2   beat_name                  464044 non-null  object
 3   isschool                   464044 non-null  int64 
 4   date_stop                  464044 non-null  object
 5   time_stop                  464044 non-null  object
 6   stopduration               464044 non-null  int64 
 7   stop_in_response_to_cfs    464044 non-null  int64 
 8   officer_assignment_key     464044 non-null  int64 
 9   assignment                 464044 non-null  object
 10  exp_years                  464044 non-null  int64 
 11  pid                        464044 non-null  int64 
 12  isstudent                  464044 non-null  int64 
 13  perceived_limited_english  464044 non-null  

## Perceived Gender

In [9]:
rdf['perceived_gender'].unique()

array(['Male', 'Female', 'Transgender woman/girl', nan,
       'Transgender man/boy'], dtype=object)

In [10]:
# Put nonbinary gender values into one bin
rdf['perceived_gender'].replace(['Transgender woman/girl', 'Transgender man/boy', np.nan], 'Other', inplace=True)

In [11]:
# Prepare for one hot encoding
rdf['perceived_gender'] = rdf['perceived_gender'].apply(lambda x: 'gender_' + x.lower())

In [12]:
# Use one hot encoder to fit and transform perceived_gender column
g_encoder =  OneHotEncoder().fit(rdf[['perceived_gender']])
g_transformed = g_encoder.transform(rdf[['perceived_gender']]).toarray()
g_encoder.categories_

[array(['gender_female', 'gender_male', 'gender_other'], dtype=object)]

In [13]:
# Attach each binary column to the dataframe
for index, category in enumerate(np.concatenate(g_encoder.categories_)):
    rdf[category] = g_transformed[:,index]

In [14]:
#rdf = rdf.drop(['perceived_gender'], axis=1)

In [15]:
rdf.head()

,stop_id,beat,beat_name,isschool,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,assignment,exp_years,pid,isstudent,perceived_limited_english,perceived_age,perceived_gender,perceived_lgbt,gender_female,gender_male,gender_other
0,2443,122,Pacific Beach 122,0,2018-07-01,00:01:37,30,0,1,"Patrol, traffic enforcement, field operations",10,1,0,0,25,gender_male,No,0.0,1.0,0.0
1,2444,121,Mission Beach 121,0,2018-07-01,00:03:34,10,0,1,"Patrol, traffic enforcement, field operations",18,1,0,0,25,gender_male,No,0.0,1.0,0.0
2,2447,822,El Cerrito 822,0,2018-07-01,00:05:43,15,1,10,Other,1,1,0,0,30,gender_male,No,0.0,1.0,0.0
3,2447,822,El Cerrito 822,0,2018-07-01,00:05:43,15,1,10,Other,1,2,0,0,30,gender_female,No,1.0,0.0,0.0
4,2448,614,Ocean Beach 614,0,2018-07-01,00:19:06,5,0,1,"Patrol, traffic enforcement, field operations",3,1,0,0,23,gender_male,No,0.0,1.0,0.0


# Assignment

In [16]:
rdf['assignment'].unique()

array(['Patrol, traffic enforcement, field operations', 'Other',
       'Gang enforcement', 'Roadblock or DUI sobriety checkpoint',
       'Investigative/detective', 'Special events', 'Task force',
       'Narcotics/vice',
       'K1-12 public school inlcuding school resource officer or school police officer',
       'Compliance check'], dtype=object)

In [17]:
# Prepare assignment dictionary for one hot encoding
assign_dict = {'Patrol, traffic enforcement, field operations': 'assignment_A',
               'Gang enforcement': 'assignment_B',
               'Compliance check': 'assignment_C',
               'Special events': 'assignment_D',
               'Roadblock or DUI sobriety checkpoint': 'assignment_E',
               'Narcotics/vice': 'assignment_F',
               'Task force': 'assignment_G',
               'K1-12 public school inlcuding school resource officer or school police officer': 'assignment_H',
               'Investigative/detective': 'assignment_I',
               'Other': 'assignment_J'}

rdf['assignment'].replace(assign_dict, inplace=True)

In [18]:
# Use one hot encoder to fit and transform assignment column
a_encoder =  OneHotEncoder().fit(rdf[['assignment']])
a_transformed = a_encoder.transform(rdf[['assignment']]).toarray()
a_encoder.categories_

[array(['assignment_A', 'assignment_B', 'assignment_C', 'assignment_D',
        'assignment_E', 'assignment_F', 'assignment_G', 'assignment_H',
        'assignment_I', 'assignment_J'], dtype=object)]

In [19]:
# Attach transformed columns to dataframe
for index, category in enumerate(np.concatenate(a_encoder.categories_)):
    rdf[category] = a_transformed[:,index]

In [20]:
rdf = rdf.drop(['assignment'], axis=1)

In [21]:
rdf.head()

,stop_id,beat,beat_name,isschool,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,exp_years,...,assignment_A,assignment_B,assignment_C,assignment_D,assignment_E,assignment_F,assignment_G,assignment_H,assignment_I,assignment_J
0,2443,122,Pacific Beach 122,0,2018-07-01,00:01:37,30,0,1,10,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2444,121,Mission Beach 121,0,2018-07-01,00:03:34,10,0,1,18,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2447,822,El Cerrito 822,0,2018-07-01,00:05:43,15,1,10,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2447,822,El Cerrito 822,0,2018-07-01,00:05:43,15,1,10,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2448,614,Ocean Beach 614,0,2018-07-01,00:19:06,5,0,1,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Stop time

In [22]:
# Really weird occurrences with ancient years in time_stop column!
weird_ids = rdf['time_stop'].apply(lambda x: (len(x.split()) == 2))
rdf[weird_ids]

,stop_id,beat,beat_name,isschool,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,exp_years,...,assignment_A,assignment_B,assignment_C,assignment_D,assignment_E,assignment_F,assignment_G,assignment_H,assignment_I,assignment_J
97194,92205,524,Core-Columbia 524,0,2019-01-18,1900-01-01 04:48:00,30,1,1,10,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113838,109883,313,Kearney Mesa 313,0,2019-02-22,1900-01-01 02:53:09,15,0,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124771,120608,722,Nestor 722,0,2019-03-16,1900-01-01 05:47:19,30,0,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127568,123099,512,Logan Heights 512,0,2019-03-22,1900-01-01 01:56:00,25,0,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138748,132994,446,Lincoln Park 446,0,2019-04-11,1900-01-01 03:33:24,45,1,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438827,397980,723,Otay Mesa West 723,0,2021-01-29,1899-12-30 00:00:00,180,0,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
443226,402207,124,La Jolla 124,0,2021-02-10,1899-12-30 00:00:00,10,0,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
443281,402252,836,Swan Canyon 836,0,2021-02-10,1899-12-30 00:00:00,90,0,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
453554,411242,834,Castle 834,0,2021-03-06,1899-12-30 00:00:00,30,0,1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Create datetime objects from time_stop values
# (taking into account the weird values)
def make_timestamp(row):
    if len(row.time_stop.split()) == 2:
        row.time_stop = row.time_stop.split()[1]
    
    timestr = row.date_stop + ' ' + row.time_stop
    
    return datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S')

In [24]:
# Create timestamp column
rdf['timestamp'] = rdf.apply(lambda row: make_timestamp(row), axis=1)

In [25]:
# Play around with timestamp
min_ts = min(rdf['timestamp'])
max_ts = max(rdf['timestamp'])
print(min_ts)
print(max_ts)
print(max(rdf['timestamp']))
print(max(rdf['timestamp']) - min(rdf['timestamp']))
print((max_ts - min_ts).total_seconds())

2018-07-01 00:01:37
2021-03-31 23:39:35
2021-03-31 23:39:35
1004 days 23:37:58
86830678.0


In [26]:
rdf.head()

,stop_id,beat,beat_name,isschool,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,exp_years,...,assignment_B,assignment_C,assignment_D,assignment_E,assignment_F,assignment_G,assignment_H,assignment_I,assignment_J,timestamp
0,2443,122,Pacific Beach 122,0,2018-07-01,00:01:37,30,0,1,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-07-01 00:01:37
1,2444,121,Mission Beach 121,0,2018-07-01,00:03:34,10,0,1,18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-07-01 00:03:34
2,2447,822,El Cerrito 822,0,2018-07-01,00:05:43,15,1,10,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-07-01 00:05:43
3,2447,822,El Cerrito 822,0,2018-07-01,00:05:43,15,1,10,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018-07-01 00:05:43
4,2448,614,Ocean Beach 614,0,2018-07-01,00:19:06,5,0,1,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-07-01 00:19:06


# Day, Month, Year

In [27]:
print(max(rdf['timestamp']).year)
print(max(rdf['timestamp']).month)
print(max(rdf['timestamp']).day)
print(max(rdf['timestamp']).weekday()) # 0 -> 6 == Monday -> Sunday

2021
3
31
2


In [28]:
# Attach separate columns for year, month, day and weekday from timestamps
rdf['year'] = rdf['timestamp'].apply(lambda ts: ts.year)
rdf['month'] = rdf['timestamp'].apply(lambda ts: ts.month)
rdf['day'] = rdf['timestamp'].apply(lambda ts: ts.day)
rdf['weekday'] = rdf['timestamp'].apply(lambda ts: ts.weekday())

In [29]:
rdf.head()

,stop_id,beat,beat_name,isschool,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,exp_years,...,assignment_F,assignment_G,assignment_H,assignment_I,assignment_J,timestamp,year,month,day,weekday
0,2443,122,Pacific Beach 122,0,2018-07-01,00:01:37,30,0,1,10,...,0.0,0.0,0.0,0.0,0.0,2018-07-01 00:01:37,2018,7,1,6
1,2444,121,Mission Beach 121,0,2018-07-01,00:03:34,10,0,1,18,...,0.0,0.0,0.0,0.0,0.0,2018-07-01 00:03:34,2018,7,1,6
2,2447,822,El Cerrito 822,0,2018-07-01,00:05:43,15,1,10,1,...,0.0,0.0,0.0,0.0,1.0,2018-07-01 00:05:43,2018,7,1,6
3,2447,822,El Cerrito 822,0,2018-07-01,00:05:43,15,1,10,1,...,0.0,0.0,0.0,0.0,1.0,2018-07-01 00:05:43,2018,7,1,6
4,2448,614,Ocean Beach 614,0,2018-07-01,00:19:06,5,0,1,3,...,0.0,0.0,0.0,0.0,0.0,2018-07-01 00:19:06,2018,7,1,6


# Number of Days and Number of Hours

In [30]:
def numDays(data):
    theDate = datetime.strptime(data, "%Y-%m-%d")
    return (theDate - datetime(theDate.year,1,1)).days + 1

In [31]:
rdf['dayofyear'] = rdf['date_stop'].apply(numDays)

In [32]:

rdf['time_stop'] = [(rdf['time_stop'][x][-8:]) for x in range(0,len(rdf))]

In [33]:
def numHours(data):
    theTime = datetime.strptime(data,"%H:%M:%S")
    return theTime.hour + theTime.minute/60. + theTime.second/3600.

In [34]:
rdf['timeofday'] = rdf['time_stop'].apply(numHours)

In [35]:
rdf["beat"] = rdf["beat"].astype('category')

In [36]:
rdf["beat_cat"] = rdf["beat"].cat.codes

In [37]:
rdf[['beat','beat_name','beat_cat','dayofyear','timeofday','stopduration','officer_assignment_key','perceived_limited_english',
         'perceived_age','perceived_gender','perceived_lgbt']].to_csv('darrendata.csv', index=False) 